In [ ]:
import numpy as np
import scipy as sp
import pandas as pd

from scipy import optimize

%matplotlib inline 
import matplotlib.pyplot as plt

from simul_functions import generate_toy_data, get_bootstrap_data, set_intensity, get_data_to_fit
from simul_functions import get_attenuation_parameters


In [ ]:
# input parameters for the simualtions
minE = 10**15 #eV
maxE = 10**18 #eV
gamma = -2.5 # slope of the spectrum

#relation between energy and s38
A = 10**12
B = 1.2

#attenuation true numbers
alpha = 0.919
beta = -1.13

#number of events to generate
events = 10000
#number of bins for the zenith 
nr_of_bins = 8

data = generate_toy_data(events, minE, maxE, gamma, A, B, alpha, beta)

#print(data)

In [ ]:
#### fit the attenuation curve
data = set_intensity(data, nr_of_bins)
#print(data)        
    
### input value the reference intesity
intensity = 100
s125_fit, bins = get_data_to_fit(data, intensity, nr_of_bins-1)

values, cov = get_attenuation_parameters(s125_fit, bins, performMCMC = False )

#print(values)


In [ ]:
### number of boostrap samples as input
samples = 2
from simul_functions import get_s125

fig2, ax2 = plt.subplots(figsize=(10,6))
plt.grid()
a=[alpha,beta,1]
bootstrap_values=[]
bootstrap_values_2=[]
for j in range(0, samples):
    new_data = get_bootstrap_data(data)
    new_data=set_intensity(new_data, nr_of_bins)
    s125_fit, bins = get_data_to_fit(new_data, intensity, nr_of_bins-1)
    vals, cov_bt = get_attenuation_parameters(s125_fit, bins, performMCMC = False )
    bootstrap_values.append(vals)
    bootstrap_values_2.append(vals**2)
    ax2.plot(bins, s125_fit, lw=0, marker='o')
    ax2.plot(bins, get_s125(bins,vals[0], vals[1], vals[2]))
    print(vals)


In [ ]:
print(bootstrap_values_2)

Things to do:
1. From the bootstrap get all the values and plot them in an histogram and one vs each other (in one canvas)
2. Give final results: values(from original data) +- rms(vals)
3. Implement MCMC fit in the get_attenuation_parameters function
4. Compare results with 2.
5. See/write to Ioana/Zig

In [ ]:
from simul_functions import bootstrap_graphs

plot=bootstrap_graphs(bootstrap_values)

In [ ]:
#from matplotlib.backends.backend_pdf import PdfPages

plot.savefig("bootstrap_values2")

In [ ]:
#final results with error
from simul_functions import rms_for_initial_vals

error=rms_for_initial_vals(bootstrap_values, bootstrap_values_2)
final_values_up=values + error
final_values_down=values - error

print(values,'+/-',error)
print(final_values)